# Guest House - Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app13-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@45e61d04

In [2]:
import spark.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@48a37a1d

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val booking = hiveCxt.table("sqlzoo.booking")
val guest = hiveCxt.table("sqlzoo.guest")
val room = hiveCxt.table("sqlzoo.room")
val rate = hiveCxt.table("sqlzoo.rate")
val extra = hiveCxt.table("sqlzoo.extra")

booking: DataFrame = [booking_id: int, booking_date: string ... 6 more fields]
guest: DataFrame = [id: int, first_name: string ... 2 more fields]
room: DataFrame = [id: int, room_type: string ... 1 more field]
rate: DataFrame = [room_type: string, occupancy: int ... 1 more field]
extra: DataFrame = [extra_id: int, booking_id: int ... 2 more fields]

## 6.
Ruth Cadbury. Show the total amount payable by guest Ruth Cadbury for her room bookings. You should JOIN to the rate table using room_type_requested and occupants.

```
+--------------------+
| SUM(nights*amount) |
+--------------------+
|             552.00 |
+--------------------+
```

In [5]:
(guest.filter((col("first_name")==="Ruth") && 
              (col("last_name")==="Cadbury"))
 .join(booking, (guest("id")===booking("guest_id")))
 .join(rate, (col("room_type_requested")===rate("room_type")) && 
             (col("occupants")===rate("occupancy")))
 .groupBy()
 .agg(sum(col("nights") * col("amount")).alias("SUM(nights*amount)"))
 .showHTML())

SUM(nights*amount)
552.0


## 7.
Including Extras. Calculate the total bill for booking 5346 including extras.

```
+-------------+
| SUM(amount) |
+-------------+
|      118.56 |
+-------------+
```

In [6]:
(booking.filter(col("booking_id")===5346)
 .join(rate, (booking("occupants")===rate("occupancy")) && 
             (booking("room_type_requested")===rate("room_type")))
 .union(booking.filter(col("booking_id")===5346)
        .join(extra, "booking_id"))
 .groupBy()
 .agg(sum("amount").alias("SUM(amount)"))
 .showHTML())

SUM(amount)
118.56


## 8.
Edinburgh Residents. For every guest who has the word “Edinburgh” in their address show the total number of nights booked. Be sure to include 0 for those guests who have never had a booking. Show last name, first name, address and number of nights. Order by last name then first name.

```
+-----------+------------+---------------------------+--------+
| last_name | first_name | address                   | nights |
+-----------+------------+---------------------------+--------+
| Brock     | Deidre     | Edinburgh North and Leith |      0 |
| Cherry    | Joanna     | Edinburgh South West      |      0 |
| Murray    | Ian        | Edinburgh South           |     13 |
| Sheppard  | Tommy      | Edinburgh East            |      0 |
| Thomson   | Michelle   | Edinburgh West            |      3 |
+-----------+------------+---------------------------+--------+
```

In [7]:
(guest.filter(col("address").like("%Edinburgh%"))
 .join(booking, (guest("id")===booking("guest_id")), joinType="left")
 .na.fill(0, Array("nights"))
 .groupBy("id", "last_name", "first_name", "address")
 .agg(sum("nights"))
 .orderBy("last_name", "first_name")
 .showHTML())

id,last_name,first_name,address,sum(nights)
1591,Brock,Deidre,Edinburgh North and Leith,0
1599,Cherry,Joanna,Edinburgh South West,0
1027,Murray,Ian,Edinburgh South,13
1617,Sheppard,Tommy,Edinburgh East,0
1457,Thomson,Michelle,Edinburgh West,3


## 9.
How busy are we? For each day of the week beginning 2016-11-25 show the number of bookings starting that day. Be sure to show all the days of the week in the correct order.

```
+------------+----------+
| i          | arrivals |
+------------+----------+
| 2016-11-25 |        7 |
| 2016-11-26 |        8 |
| 2016-11-27 |       12 |
| 2016-11-28 |        7 |
| 2016-11-29 |       13 |
| 2016-11-30 |        6 |
| 2016-12-01 |        7 |
+------------+----------+
```

In [8]:
(booking.filter((col("booking_date") >= "2016-11-25") && 
                (col("booking_date") <= date_add(lit("2016-11-25"), lit(6))))
 .groupBy("booking_date")
 .agg(count("booking_id").alias("arrivals"))
 .orderBy("booking_date")
 .showHTML())

booking_date,arrivals
2016-11-25,7
2016-11-26,8
2016-11-27,12
2016-11-28,7
2016-11-29,13
2016-11-30,6
2016-12-01,7


## 10.
How many guests? Show the number of guests in the hotel on the night of 2016-11-21. Include all occupants who checked in that day but not those who checked out.

```
+----------------+
| SUM(occupants) |
+----------------+
|             39 |
+----------------+
```

In [9]:
(booking.filter((col("booking_date") <= "2016-11-21") &&
                (date_add(col("booking_date"), col("nights")) > "2016-11-21"))
 .groupBy()
 .agg(sum("occupants").alias("SUM(occupants"))
 .showHTML())

SUM(occupants
39


In [10]:
spark.stop()